# Count Vect Modeling for Pros and Cons

# Pros Modeling 

In [12]:
import wrangle as w
import nltk
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams


In [2]:
train, val, test = w.wrangle_glassdoor()

In [3]:
train.head(3)

,pros,cons,name,rating,ceo_approval,friend_recommendation,pros_cleaned,pros_lemmatized,cons_cleaned,cons_lemmatized,binned_rating,binned_rating_int
url,,,,,,,,,,,,
https://www.glassdoor.com/Reviews/Perficient-Reviews-E9329.htm,Perficient is an ethical company that actually...,"None at all, love, love, love this company!\nI...",Perficient,4.1,87.0,80.0,perficient is an ethical company that actually...,perficient ethical company actually value empl...,none at all love love love this company\nit is...,none love love love company good company canno...,Four,4
https://www.glassdoor.com/Reviews/Farmers-Insurance-Group-Reviews-E3955.htm,"This company is the best ever.\nLarge, establi...",I have nothing bad to say.\nManagement company...,Farmers Insurance Group,3.4,37.0,52.0,this company is the best ever\nlarge establish...,company best ever large established company so...,i have nothing bad to say\nmanagement company ...,nothing bad say management company get paid re...,Three,3
https://www.glassdoor.com/Reviews/MIT-Reviews-E2889.htm,"Very inspiring place to work at, to feel that ...",Depends on the project to how much organizatio...,MIT,4.4,90.0,86.0,very inspiring place to work at to feel that s...,inspiring place work feel something new happen...,depends on the project to how much organizatio...,depends project much organization team include...,Four,4


In [4]:
most_common = train["binned_rating_int"].value_counts().idxmax()
most_common_frequency = train["binned_rating_int"].value_counts().max()

baseline_accuracy = most_common_frequency / len(train)

print(f"Most Common Class: {most_common}")
print(f"Baseline Accuracy: {baseline_accuracy:.2f}")



Most Common Class: 3
Baseline Accuracy: 0.68


In [29]:
df = train

vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(df["pros_lemmatized"])

word_frequencies_df = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())

word_frequencies_total = word_frequencies_df.sum()

top_n = 80
top_words = word_frequencies_total.sort_values(ascending=False).index[:top_n]

top_words
        

Index(['work', 'great', 'good', 'benefit', 'people', 'company', 'pay',
       'opportunity', 'job', 'environment', 'team', 'culture', 'lot',
       'employee', 'time', 'flexible', 'balance', 'nice', 'working', 'get',
       'management', 'hour', 'place', 'life', 'coworkers', 'schedule',
       'decent', 'easy', 'experience', 'manager', 'training', 'fun', 'growth',
       'friendly', 'really', 'day', 'learn', 'career', 'make', 'well', 'best',
       'amazing', 'like', 'care', 'free', 'discount', 'customer', 'year',
       'pto', 'many', 'new', 'home', 'leadership', 'remote', 'flexibility',
       'salary', 'help', 'staff', 'always', 'worklife', 'support', 'health',
       'one', 'learning', 'supportive', 'excellent', 'worked', 'project',
       'grow', 'office', 'paid', 'also', 'bonus', 'love', 'insurance',
       'product', 'part', 'everyone', 'pretty', 'want'],
      dtype='object')

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import ttest_ind



# Define a list of top words
top_words = ['work', 'great', 'good', 'benefit', 'people', 'company', 'pay',
             'opportunity', 'job', 'environment', 'team', 'culture', 'lot',
             'employee', 'time', 'flexible', 'balance', 'nice', 'working', 'get',
             'management', 'hour', 'place', 'life', 'coworkers', 'schedule',
             'decent', 'easy', 'experience', 'manager', 'training', 'fun', 'growth',
             'friendly', 'really', 'day', 'learn', 'career', 'make', 'well', 'best',
             'amazing', 'like', 'care', 'free', 'discount', 'customer', 'year',
             'pto', 'many', 'new', 'home', 'leadership', 'remote', 'flexibility',
             'salary', 'help', 'staff', 'always', 'worklife', 'support', 'health',
             'one', 'learning', 'supportive', 'excellent', 'worked', 'project',
             'grow', 'office', 'paid', 'also', 'bonus', 'love', 'insurance',
             'product', 'part', 'everyone', 'pretty', 'want']

# Initialize CountVectorizer
vectorizer = CountVectorizer(vocabulary=top_words, binary=True)

# Transform the text data into a binary matrix of word occurrences
word_occurrences = vectorizer.transform(df['pros_lemmatized'])

# Perform t-test for each word
results = []

for i, word in enumerate(top_words):
    # Extract the binary word occurrences for the current word
    word_occurrence = word_occurrences[:, i].toarray().flatten()

    # Split data into two groups based on the presence/absence of the word
    word_present = df[word_occurrence == 1]['binned_rating_int']
    word_absent = df[word_occurrence == 0]['binned_rating_int']

    # Perform t-test
    statistic, p_value = ttest_ind(word_present, word_absent)

    if p_value < 0.05:
        results.append({
            'Word': word,
            'T-statistic': statistic,
            'P-value': p_value,
        })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

sorted_results_df = results_df.sort_values(by='P-value')

sorted_results_df

,Word,T-statistic,P-value
28,worklife,6.428224,2.662284e-10
34,project,5.713017,1.760341e-08
9,easy,-5.504217,5.532738e-08
19,customer,-5.185511,2.963119e-07
8,decent,-4.881845,1.354138e-06
18,discount,-4.866502,1.459168e-06
2,balance,4.821641,1.813257e-06
33,excellent,4.617184,4.775617e-06
7,life,4.535524,6.960679e-06
0,company,-4.409215,1.232590e-05


## Decision Tree

In [37]:
significant_words = ['worklife', 'project', 'easy', 'customer', 'decent']

for word in significant_words:
    train[word] = train["pros_lemmatized"].apply(lambda x: x.count(word))
    
for word in significant_words:
    val[word] = val["pros_lemmatized"].apply(lambda x: x.count(word))
    
for word in significant_words:
    test[word] = test["pros_lemmatized"].apply(lambda x: x.count(word))



X_train = train[significant_words]
y_train = train["binned_rating_int"]

X_val = val[significant_words]
y_val = val["binned_rating_int"]

X_test = test[significant_words]
y_test = test["binned_rating_int"]

In [38]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")

Train Accuracy: 0.99
Validation Accuracy: 0.63


# RandomForrest

In [39]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")



Train Accuracy: 0.99
Validation Accuracy: 0.71


# KNN

In [40]:
k = 3  
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")





Train Accuracy: 0.82
Validation Accuracy: 0.67


# LogiReg

In [41]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")


Train Accuracy: 0.73
Validation Accuracy: 0.67


# Test Score for RF

In [42]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_test_pred = clf.predict(X_test)

# Calculate test accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.702020202020202


# Cons Modeling 

In [12]:
import wrangle as w
import nltk
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams


In [2]:
train, val, test = w.wrangle_glassdoor()

In [3]:
train.head(3)

,pros,cons,name,rating,ceo_approval,friend_recommendation,pros_cleaned,pros_lemmatized,cons_cleaned,cons_lemmatized,binned_rating,binned_rating_int
url,,,,,,,,,,,,
https://www.glassdoor.com/Reviews/Perficient-Reviews-E9329.htm,Perficient is an ethical company that actually...,"None at all, love, love, love this company!\nI...",Perficient,4.1,87.0,80.0,perficient is an ethical company that actually...,perficient ethical company actually value empl...,none at all love love love this company\nit is...,none love love love company good company canno...,Four,4
https://www.glassdoor.com/Reviews/Farmers-Insurance-Group-Reviews-E3955.htm,"This company is the best ever.\nLarge, establi...",I have nothing bad to say.\nManagement company...,Farmers Insurance Group,3.4,37.0,52.0,this company is the best ever\nlarge establish...,company best ever large established company so...,i have nothing bad to say\nmanagement company ...,nothing bad say management company get paid re...,Three,3
https://www.glassdoor.com/Reviews/MIT-Reviews-E2889.htm,"Very inspiring place to work at, to feel that ...",Depends on the project to how much organizatio...,MIT,4.4,90.0,86.0,very inspiring place to work at to feel that s...,inspiring place work feel something new happen...,depends on the project to how much organizatio...,depends project much organization team include...,Four,4


In [4]:
most_common = train["binned_rating_int"].value_counts().idxmax()
most_common_frequency = train["binned_rating_int"].value_counts().max()

baseline_accuracy = most_common_frequency / len(train)

print(f"Most Common Class: {most_common}")
print(f"Baseline Accuracy: {baseline_accuracy:.2f}")



Most Common Class: 3
Baseline Accuracy: 0.68


In [43]:
df = train

vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(df["cons_lemmatized"])

word_frequencies_df = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())

word_frequencies_total = word_frequencies_df.sum()

top_n = 80
top_words = word_frequencies_total.sort_values(ascending=False).index[:top_n]

top_words
        

Index(['work', 'pay', 'management', 'time', 'company', 'get', 'hour',
       'employee', 'job', 'people', 'manager', 'lot', 'year', 'day', 'like',
       'low', 'team', 'make', 'working', 'long', 'many', 'customer', 'much',
       'good', 'one', 'even', 'leadership', 'con', 'hard', 'would', 'training',
       'need', 'lack', 'high', 'could', 'change', 'new', 'sometimes', 'salary',
       'bad', 'take', 'great', 'thing', 'position', 'really', 'poor', 'little',
       'balance', 'benefit', 'always', 'life', 'better', 'go', 'growth',
       'week', 'care', 'store', 'sale', 'every', 'raise', 'also', 'culture',
       'dont', 'enough', 'never', 'opportunity', 'nothing', 'environment',
       'call', 'experience', 'place', 'want', 'office', 'way', 'staff',
       'difficult', 'know', 'none', 'shift', 'think'],
      dtype='object')

In [45]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import ttest_ind



# Define a list of top words
top_words = ['work', 'pay', 'management', 'time', 'company', 'get', 'hour',
       'employee', 'job', 'people', 'manager', 'lot', 'year', 'day', 'like',
       'low', 'team', 'make', 'working', 'long', 'many', 'customer', 'much',
       'good', 'one', 'even', 'leadership', 'con', 'hard', 'would', 'training',
       'need', 'lack', 'high', 'could', 'change', 'new', 'sometimes', 'salary',
       'bad', 'take', 'great', 'thing', 'position', 'really', 'poor', 'little',
       'balance', 'benefit', 'always', 'life', 'better', 'go', 'growth',
       'week', 'care', 'store', 'sale', 'every', 'raise', 'also', 'culture',
       'dont', 'enough', 'never', 'opportunity', 'nothing', 'environment',
       'call', 'experience', 'place', 'want', 'office', 'way', 'staff',
       'difficult', 'know', 'none', 'shift', 'think']

# Initialize CountVectorizer
vectorizer = CountVectorizer(vocabulary=top_words, binary=True)

# Transform the text data into a binary matrix of word occurrences
word_occurrences = vectorizer.transform(df['cons_lemmatized'])

# Perform t-test for each word
results = []

for i, word in enumerate(top_words):
    # Extract the binary word occurrences for the current word
    word_occurrence = word_occurrences[:, i].toarray().flatten()

    # Split data into two groups based on the presence/absence of the word
    word_present = df[word_occurrence == 1]['binned_rating_int']
    word_absent = df[word_occurrence == 0]['binned_rating_int']

    # Perform t-test
    statistic, p_value = ttest_ind(word_present, word_absent)

    if p_value < 0.05:
        results.append({
            'Word': word,
            'T-statistic': statistic,
            'P-value': p_value,
        })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

sorted_results_df = results_df.sort_values(by='P-value')

sorted_results_df

,Word,T-statistic,P-value
23,store,-7.572992,1.411566e-13
31,call,-7.292816,9.799757e-13
22,care,-5.999692,3.447267e-09
26,raise,-5.770554,1.275879e-08
39,shift,-5.623192,2.893771e-08
9,even,-5.418691,8.752464e-08
35,staff,-5.261690,1.999401e-07
7,customer,-5.200924,2.737491e-07
28,dont,-4.989770,7.961784e-07
20,go,-4.712007,3.061337e-06


## Decision Tree

In [46]:
significant_words = ['store', 'call', 'care', 'raise', 'shift']

for word in significant_words:
    train[word] = train["pros_lemmatized"].apply(lambda x: x.count(word))
    
for word in significant_words:
    val[word] = val["pros_lemmatized"].apply(lambda x: x.count(word))
    
for word in significant_words:
    test[word] = test["pros_lemmatized"].apply(lambda x: x.count(word))



X_train = train[significant_words]
y_train = train["binned_rating_int"]

X_val = val[significant_words]
y_val = val["binned_rating_int"]

X_test = test[significant_words]
y_test = test["binned_rating_int"]

In [47]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")

Train Accuracy: 0.89
Validation Accuracy: 0.54


# RandomForrest

In [48]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")



Train Accuracy: 0.89
Validation Accuracy: 0.59


# KNN

In [49]:
k = 3  
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")





Train Accuracy: 0.78
Validation Accuracy: 0.56


# LogiReg

In [50]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)

train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Validation Accuracy: {val_accuracy:.2f}")


Train Accuracy: 0.70
Validation Accuracy: 0.62


# Test Score for Logi

In [52]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_test_pred = clf.predict(X_test)

# Calculate test accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.6717171717171717
